In [1]:
import sys
import os
sys.path.append('../dags')

from database import create_database_engine


# 환경 변수 설정
os.environ["POSTGRES_USER"] = "airflow"
os.environ["POSTGRES_PASSWORD"] = "airflow"
os.environ["POSTGRES_DB"] = "events"
os.environ["POSTGRES_PORT"] = "5433"

engine = create_database_engine(host="127.0.0.1")
print("Successfully connected to the database")

Successfully connected to the database


In [2]:
import pandas as pd
from datetime import datetime

# 나머지는 변수로 관리
label_table = "kind"
return_table = "abnormal_return_kind_log"
label_col = "label"

# abnormal return 컬럼
abn_return_cols = (
    [f"abn_ret_minus_{i}m" for i in range(59, 0, -1)] +
    ["abn_ret_0m"] +
    [f"abn_ret_{i}m" for i in range(1, 61)]
)

# SQL 컬럼 문자열을 만듬 (event_id, event_ts는 하드코딩)
sql_columns = (
    [f"ar.event_ts"] +
    [f"ar.{col}" for col in abn_return_cols] +
    [f"k.{label_col}"]
)
sql_columns_str = ",\n    ".join(sql_columns)

# 2021-01-01 ~ 2022-06-30(포함) event_ts 조건 추가
query = f"""
SELECT {sql_columns_str}
FROM {return_table} ar
JOIN "{label_table}" k ON ar.event_id = k.id
WHERE ar.event_ts >= '2021-01-01' AND ar.event_ts <= '2022-06-30'
ORDER BY ar.event_ts ASC
"""

df_before_telegram = pd.read_sql(query, engine)
df_before_telegram["period_dummy"] = 0
df_before_telegram

DETAIL:  The database was created using collation version 2.36, but the operating system provides version 2.41.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE events REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.


,event_ts,abn_ret_minus_59m,abn_ret_minus_58m,abn_ret_minus_57m,abn_ret_minus_56m,abn_ret_minus_55m,abn_ret_minus_54m,abn_ret_minus_53m,abn_ret_minus_52m,abn_ret_minus_51m,...,abn_ret_53m,abn_ret_54m,abn_ret_55m,abn_ret_56m,abn_ret_57m,abn_ret_58m,abn_ret_59m,abn_ret_60m,label,period_dummy
0,2021-01-04 10:08:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.000693,0.000327,0.000981,-0.002043,0.002370,-0.000327,0.000616,-0.000982,0,0
1,2021-01-04 10:08:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.000693,0.000327,0.000981,-0.002043,0.002370,-0.000327,0.000616,-0.000982,0,0
2,2021-01-04 10:08:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000327,0.000327,0.000981,0.000000,-0.004539,-0.000327,0.006503,-0.005849,1,0
3,2021-01-04 10:14:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000584,-0.007325,-0.004989,0.001311,-0.002462,0.000328,0.000085,0.004667,1,0
4,2021-01-04 10:16:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.002033,-0.005333,-0.000124,0.000373,-0.000995,-0.000373,0.000000,-0.003549,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5067,2022-06-29 14:04:00+00:00,0.004396,-0.004396,0.008295,-0.004376,0.000795,-0.000318,-0.000318,-0.004713,0.000159,...,-0.004080,0.004397,-0.000159,0.000635,-0.009132,0.000000,0.004577,-0.003941,1,0
5068,2022-06-29 14:20:00+00:00,-0.001768,0.000000,-0.000159,0.000477,-0.000318,0.002245,-0.001291,0.000159,-0.000477,...,-0.000159,-0.001291,0.000318,0.003693,-0.000159,0.001921,-0.004977,0.000000,1,0
5069,2022-06-29 14:21:00+00:00,-0.001219,-0.000159,0.000477,-0.000318,-0.000743,0.000477,-0.001063,0.001965,0.000159,...,0.000477,0.000318,-0.001060,0.001060,0.000159,0.000318,0.000000,NaN,1,0
5070,2022-06-29 14:35:00+00:00,-0.000635,-0.000962,0.001914,0.000801,-0.001277,0.001594,-0.000476,-0.000159,-0.000476,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0


In [3]:
label_table = "label"
return_table = "abnormal_return_log"
label_col = "label"


all_sql_columns = (
    [f"ar.event_ts"] +
    [f"ar.{col}" for col in abn_return_cols] +
    [f"k.{label_col}"] +
    ["k.report_name"] +
    ["k.summary_kr"]
)
all_sql_columns_str = ",\n    ".join(all_sql_columns)

query = f"""
SELECT {all_sql_columns_str}
FROM {return_table} ar
JOIN "{label_table}" k ON ar.event_id = k.id
JOIN disclosure_events de ON ar.event_id = de.id
WHERE k.{label_col} IS NOT NULL
ORDER BY ar.event_ts ASC
"""

df_after_telegram = pd.read_sql(query, engine)
df_after_telegram["period_dummy"] = 1
label_map = {0: 1, 1: 0, 2: -1}
df_after_telegram['label'] = df_after_telegram['label'].map(label_map)
df_after_telegram

,event_ts,abn_ret_minus_59m,abn_ret_minus_58m,abn_ret_minus_57m,abn_ret_minus_56m,abn_ret_minus_55m,abn_ret_minus_54m,abn_ret_minus_53m,abn_ret_minus_52m,abn_ret_minus_51m,...,abn_ret_55m,abn_ret_56m,abn_ret_57m,abn_ret_58m,abn_ret_59m,abn_ret_60m,label,report_name,summary_kr,period_dummy
0,2022-07-01 01:03:17+00:00,-0.007148,-0.005285,0.000471,-0.012082,0.008214,-0.002471,0.000944,-0.006312,0.002364,...,0.000000,-0.001969,0.007349,-0.000450,0.004910,-0.000482,1,타법인주식및출자증권취득결정(자율공시),"강원에너지(시가총액 1,842억 원)는 2022년 7월 1일 강원이솔루션의 주식을 ...",1
1,2022-07-01 01:24:45+00:00,0.000476,0.000476,-0.007229,-0.000477,-0.002379,-0.001425,-0.000949,0.008139,-0.000476,...,-0.000965,0.000483,-0.000965,0.001931,-0.000483,-0.000483,1,단일판매ㆍ공급계약체결,비츠로셀이 2022년 6월 30일부터 2023년 10월 20일까지 방위사업청과 리튬...,1
2,2022-07-01 01:27:02+00:00,-0.000477,-0.002379,0.000009,0.003340,0.002373,-0.000476,-0.002376,-0.001904,0.000468,...,0.001931,-0.000483,-0.000483,0.000483,-0.000483,-0.000483,1,단일판매ㆍ공급계약체결(자회사의 주요경영사항),비츠로테크는 방위사업청과 리튬전지류 및 기뢰정비장비용 전지조립체 공급 계약을 체결했...,1
3,2022-07-01 01:43:15+00:00,-0.000162,-0.001130,-0.000968,0.000322,-0.000967,-0.000644,0.000161,-0.003180,0.000483,...,0.002006,0.000326,-0.000489,-0.001517,-0.000163,0.000815,1,신규시설투자등(자율공시),동원시스템즈는 2022년 7월 1일부터 2023년 7월 31일까지 585억 원을 신...,1
4,2022-07-01 02:30:09+00:00,-0.004003,-0.002731,-0.004034,0.000000,-0.009572,-0.000842,0.001445,0.000000,0.007185,...,0.000486,-0.003247,0.004219,0.000887,-0.002830,0.004288,-1,투자판단관련주요경영사항(노인성 황반변성 환자에서 OLX10212의 안전성 및 내약성...,올릭스는 노인성 황반변성 치료를 위한 OLX10212의 제1상 임상시험을 미국에서 ...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8758,2023-12-28 05:53:08+00:00,-0.000883,0.000139,-0.003026,-0.000327,-0.000418,-0.001025,-0.000279,-0.000418,0.000139,...,NaN,NaN,NaN,NaN,NaN,NaN,1,단일판매ㆍ공급계약체결,까뮤이앤씨는 조달청과 전라남도 목포시에서 수산식품 수출단지 조성사업 건축공사(공급지...,1
8759,2023-12-28 05:55:03+00:00,0.001045,0.000279,-0.001464,-0.000941,-0.003423,-0.002520,0.001716,0.000525,0.001574,...,NaN,NaN,NaN,NaN,NaN,NaN,0,조회공시요구(현저한시황변동)에대한답변(미확정),"에코프로머티리얼즈는 신규 고객 확보와 관련된 사업 협의를 진행 중이며, 아직 확정된...",1
8760,2023-12-28 05:56:27+00:00,-0.003261,0.001214,0.004819,-0.001925,-0.002289,0.000000,-0.000966,-0.000360,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,1,[기재정정]연결재무제표기준영업실적등에대한전망(공정공시),"에코마케팅은 2023년 매출액 3,500억 원과 영업이익 530억 원을 기록할 것으...",1
8761,2023-12-28 05:56:32+00:00,-0.003261,0.001214,0.004819,-0.001925,-0.002289,0.000000,-0.000966,-0.000360,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,1,[기재정정]영업실적등에대한전망(공정공시),"에코마케팅은 2023년 매출액이 490억 원, 영업이익이 200억 원으로 예상되며,...",1


In [4]:
from config import keywords
import re
import os

def classify_report_name_with_keywords_return_series_and_list(df, keywords):
    """
    df: DataFrame, must include 'report_name' column
    keywords: dict, key=disclosure_type, value=[regex strings]
    
    Returns: (Series of classified disclosure_type, List of disclosure_types in order of df)
    """
    pattern_list = []
    for disclosure_type, pattern_strs in keywords.items():
        if pattern_strs:
            for pat in pattern_strs:
                pattern_list.append( (disclosure_type, re.compile(pat)) )
        else:
            pattern_list.append( (disclosure_type, re.compile(disclosure_type)) )

    def classify_single_report(report_name):
        for disclosure_type, pat in pattern_list:
            if isinstance(report_name, str):
                if pat.search(report_name):
                    return disclosure_type
        return None

    result_list = df["report_name"].apply(classify_single_report).tolist()
    return pd.Series(result_list, index=df.index), result_list

output_dir = "type"
os.makedirs(output_dir, exist_ok=True)

# 분류도 csv로 저장하고 df_after_telegram['disclosure_type']에도 넣기
disclosure_type_series, disclosure_type_list = classify_report_name_with_keywords_return_series_and_list(
    df_after_telegram, keywords)
df_after_telegram["disclosure_type"] = disclosure_type_series

# 각 event type(disclosure_type) 별로 따로 csv 저장 (type 폴더에!)
for dtype, group in df_after_telegram.groupby("disclosure_type"):
    if dtype is None:
        continue  # 기타 처리로 분리
    fname = f"{dtype}.csv".replace("/", "_").replace("\\", "_")
    path = os.path.join(output_dir, fname)
    # summary_kr도 같이 저장 (없으면 없는 컬럼으로 저장)
    cols_to_save = ["report_name"]
    if "summary_kr" in group.columns:
        cols_to_save.append("summary_kr")
    group[cols_to_save].to_csv(path, index=False)

# 분류되지 않은 event는 "기타.csv"로 저장
not_classified = df_after_telegram[df_after_telegram['disclosure_type'].isna()]
if not not_classified.empty:
    etc_path = os.path.join(output_dir, "기타.csv")
    cols_to_save = ["report_name"]
    if "summary_kr" in not_classified.columns:
        cols_to_save.append("summary_kr")
    not_classified[cols_to_save].to_csv(etc_path, index=False)


In [5]:
from config import keywords

# 제외할 disclosure_type 목록
exclude_types = ["실적공시", "지분공시", "IR활동", "배당"]

# disclosure_type이 keywords key에 있으면서 exclude_types에는 없는 값만 보여주기
filtered_types = [k for k in keywords.keys() if k not in exclude_types]
df_after_telegram[df_after_telegram["disclosure_type"].isin(filtered_types)]["disclosure_type"].value_counts()

disclosure_type
단일판매ㆍ공급계약체결           1054
매출액변동                  351
자기주식취득 신탁계약 체결 결정      205
타법인 주식 및 출자증권 양수결정     110
자기주식 취득 결정              75
특허권취득                   74
자기주식 처분 결정              68
신규시설투자                  67
유상증자 결정                 62
임상 계획 신청                51
무상증자 결정                 49
소송등의판결ㆍ결정               28
임상 계획 승인                27
전환사채권 발행결정              22
자기주식 소각 결정              21
타법인 주식 및 출자증권 양도결정      20
소송 등의 제기                19
유형자산 양수 결정              11
회사분할 결정                  9
회사합병 결정                  9
품목허가 신청                  9
생산중단                     7
임상 계획 결과 발표              7
감자 결정                    4
자기주식취득 신탁계약 해지 결정        3
품목허가 승인                  3
임상 계획 철회                 3
기술이전계약체결                 3
영업양도 결정                  2
영업양수 결정                  2
품목허가 철회                  2
주식교환ㆍ이전 결정               1
Name: count, dtype: int64

In [6]:
df_after_telegram_filtered = df_after_telegram[df_after_telegram["disclosure_type"].isin(filtered_types)]
df_after_telegram_filtered

,event_ts,abn_ret_minus_59m,abn_ret_minus_58m,abn_ret_minus_57m,abn_ret_minus_56m,abn_ret_minus_55m,abn_ret_minus_54m,abn_ret_minus_53m,abn_ret_minus_52m,abn_ret_minus_51m,...,abn_ret_56m,abn_ret_57m,abn_ret_58m,abn_ret_59m,abn_ret_60m,label,report_name,summary_kr,period_dummy,disclosure_type
0,2022-07-01 01:03:17+00:00,-0.007148,-0.005285,0.000471,-0.012082,0.008214,-0.002471,0.000944,-0.006312,0.002364,...,-0.001969,0.007349,-0.000450,0.004910,-0.000482,1,타법인주식및출자증권취득결정(자율공시),"강원에너지(시가총액 1,842억 원)는 2022년 7월 1일 강원이솔루션의 주식을 ...",1,타법인 주식 및 출자증권 양수결정
1,2022-07-01 01:24:45+00:00,0.000476,0.000476,-0.007229,-0.000477,-0.002379,-0.001425,-0.000949,0.008139,-0.000476,...,0.000483,-0.000965,0.001931,-0.000483,-0.000483,1,단일판매ㆍ공급계약체결,비츠로셀이 2022년 6월 30일부터 2023년 10월 20일까지 방위사업청과 리튬...,1,단일판매ㆍ공급계약체결
2,2022-07-01 01:27:02+00:00,-0.000477,-0.002379,0.000009,0.003340,0.002373,-0.000476,-0.002376,-0.001904,0.000468,...,-0.000483,-0.000483,0.000483,-0.000483,-0.000483,1,단일판매ㆍ공급계약체결(자회사의 주요경영사항),비츠로테크는 방위사업청과 리튬전지류 및 기뢰정비장비용 전지조립체 공급 계약을 체결했...,1,단일판매ㆍ공급계약체결
3,2022-07-01 01:43:15+00:00,-0.000162,-0.001130,-0.000968,0.000322,-0.000967,-0.000644,0.000161,-0.003180,0.000483,...,0.000326,-0.000489,-0.001517,-0.000163,0.000815,1,신규시설투자등(자율공시),동원시스템즈는 2022년 7월 1일부터 2023년 7월 31일까지 585억 원을 신...,1,신규시설투자
4,2022-07-01 02:30:09+00:00,-0.004003,-0.002731,-0.004034,0.000000,-0.009572,-0.000842,0.001445,0.000000,0.007185,...,-0.003247,0.004219,0.000887,-0.002830,0.004288,-1,투자판단관련주요경영사항(노인성 황반변성 환자에서 OLX10212의 안전성 및 내약성...,올릭스는 노인성 황반변성 치료를 위한 OLX10212의 제1상 임상시험을 미국에서 ...,1,임상 계획 신청
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8751,2023-12-28 04:10:54+00:00,-0.000417,-0.000139,0.000139,0.000000,0.000000,0.000000,-0.000417,0.000139,-0.000139,...,-0.000139,-0.003716,0.003159,0.000000,-0.003284,1,단일판매ㆍ공급계약체결,에쓰씨엔지니어링은 2023년 12월 28일 전북 군산시에서 NEXEON Korea와...,1,단일판매ㆍ공급계약체결
8754,2023-12-28 04:54:23+00:00,0.000720,0.003656,0.000360,-0.004016,0.000000,0.000360,0.000360,0.004759,-0.005119,...,0.001103,0.001100,0.002176,-0.000360,0.000719,1,기타경영사항(자율공시)(펩타이드함유 더말 필러(레보필울트라)의 주름개선 효과와 안전...,케어젠은 펩타이드 함유 더말 필러(레보필 울트라)의 주름개선 효과와 안전성을 확인하...,1,임상 계획 승인
8756,2023-12-28 05:42:24+00:00,0.000000,0.000000,-0.000139,0.000279,-0.000279,0.000279,-0.000418,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,1,단일판매ㆍ공급계약체결,한창제지는 케이티앤지와 내자 아이보리판지 구매를 위한 455억 원 규모의 판매·공급...,1,단일판매ㆍ공급계약체결
8757,2023-12-28 05:47:03+00:00,0.000000,-0.000946,0.000585,0.000361,-0.000721,0.001305,-0.002251,0.000361,0.000000,...,NaN,NaN,NaN,NaN,NaN,1,단일판매ㆍ공급계약체결,윤성에프앤씨는 2023년 12월 27일부터 2024년 11월 16일까지 10개월간 ...,1,단일판매ㆍ공급계약체결


In [7]:
df_total = pd.concat([df_before_telegram, df_after_telegram_filtered])



In [8]:
df_total

,event_ts,abn_ret_minus_59m,abn_ret_minus_58m,abn_ret_minus_57m,abn_ret_minus_56m,abn_ret_minus_55m,abn_ret_minus_54m,abn_ret_minus_53m,abn_ret_minus_52m,abn_ret_minus_51m,...,abn_ret_56m,abn_ret_57m,abn_ret_58m,abn_ret_59m,abn_ret_60m,label,period_dummy,report_name,summary_kr,disclosure_type
0,2021-01-04 10:08:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.002043,0.002370,-0.000327,0.000616,-0.000982,0,0,NaN,NaN,NaN
1,2021-01-04 10:08:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.002043,0.002370,-0.000327,0.000616,-0.000982,0,0,NaN,NaN,NaN
2,2021-01-04 10:08:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,-0.004539,-0.000327,0.006503,-0.005849,1,0,NaN,NaN,NaN
3,2021-01-04 10:14:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.001311,-0.002462,0.000328,0.000085,0.004667,1,0,NaN,NaN,NaN
4,2021-01-04 10:16:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000373,-0.000995,-0.000373,0.000000,-0.003549,-1,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8751,2023-12-28 04:10:54+00:00,-0.000417,-0.000139,0.000139,0.000000,0.000000,0.000000,-0.000417,0.000139,-0.000139,...,-0.000139,-0.003716,0.003159,0.000000,-0.003284,1,1,단일판매ㆍ공급계약체결,에쓰씨엔지니어링은 2023년 12월 28일 전북 군산시에서 NEXEON Korea와...,단일판매ㆍ공급계약체결
8754,2023-12-28 04:54:23+00:00,0.000720,0.003656,0.000360,-0.004016,0.000000,0.000360,0.000360,0.004759,-0.005119,...,0.001103,0.001100,0.002176,-0.000360,0.000719,1,1,기타경영사항(자율공시)(펩타이드함유 더말 필러(레보필울트라)의 주름개선 효과와 안전...,케어젠은 펩타이드 함유 더말 필러(레보필 울트라)의 주름개선 효과와 안전성을 확인하...,임상 계획 승인
8756,2023-12-28 05:42:24+00:00,0.000000,0.000000,-0.000139,0.000279,-0.000279,0.000279,-0.000418,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,1,1,단일판매ㆍ공급계약체결,한창제지는 케이티앤지와 내자 아이보리판지 구매를 위한 455억 원 규모의 판매·공급...,단일판매ㆍ공급계약체결
8757,2023-12-28 05:47:03+00:00,0.000000,-0.000946,0.000585,0.000361,-0.000721,0.001305,-0.002251,0.000361,0.000000,...,NaN,NaN,NaN,NaN,NaN,1,1,단일판매ㆍ공급계약체결,윤성에프앤씨는 2023년 12월 27일부터 2024년 11월 16일까지 10개월간 ...,단일판매ㆍ공급계약체결


In [9]:
for m in range(10, 61, 10):
    plus_cols = [
        f"abn_ret_minus_{abs(i)}m" if i < 0 else f"abn_ret_{i}m"
        for i in range(-m+1, m + 1)
    ]
    df_total[f"CAR_{m}m"] = df_total[plus_cols].sum(axis=1)

In [10]:
df_total

,event_ts,abn_ret_minus_59m,abn_ret_minus_58m,abn_ret_minus_57m,abn_ret_minus_56m,abn_ret_minus_55m,abn_ret_minus_54m,abn_ret_minus_53m,abn_ret_minus_52m,abn_ret_minus_51m,...,period_dummy,report_name,summary_kr,disclosure_type,CAR_10m,CAR_20m,CAR_30m,CAR_40m,CAR_50m,CAR_60m
0,2021-01-04 10:08:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,0.016961,0.023220,0.021990,0.022972,0.019986,0.023341
1,2021-01-04 10:08:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,0.016961,0.023220,0.021990,0.022972,0.019986,0.023341
2,2021-01-04 10:08:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,-0.004914,0.007855,-0.000312,0.000670,0.008463,0.004906
3,2021-01-04 10:14:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,0.037109,0.039253,0.046965,0.025485,0.029299,0.020446
4,2021-01-04 10:16:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,-0.001911,-0.023724,-0.020824,-0.024954,-0.021079,-0.033610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8751,2023-12-28 04:10:54+00:00,-0.000417,-0.000139,0.000139,0.000000,0.000000,0.000000,-0.000417,0.000139,-0.000139,...,1,단일판매ㆍ공급계약체결,에쓰씨엔지니어링은 2023년 12월 28일 전북 군산시에서 NEXEON Korea와...,단일판매ㆍ공급계약체결,0.025168,0.047550,0.034067,0.029343,0.035203,0.028985
8754,2023-12-28 04:54:23+00:00,0.000720,0.003656,0.000360,-0.004016,0.000000,0.000360,0.000360,0.004759,-0.005119,...,1,기타경영사항(자율공시)(펩타이드함유 더말 필러(레보필울트라)의 주름개선 효과와 안전...,케어젠은 펩타이드 함유 더말 필러(레보필 울트라)의 주름개선 효과와 안전성을 확인하...,임상 계획 승인,0.002595,0.008786,0.006231,0.005858,0.001832,0.009477
8756,2023-12-28 05:42:24+00:00,0.000000,0.000000,-0.000139,0.000279,-0.000279,0.000279,-0.000418,0.000000,0.000000,...,1,단일판매ㆍ공급계약체결,한창제지는 케이티앤지와 내자 아이보리판지 구매를 위한 455억 원 규모의 판매·공급...,단일판매ㆍ공급계약체결,-0.003717,-0.001558,-0.003776,-0.007049,-0.007085,-0.007642
8757,2023-12-28 05:47:03+00:00,0.000000,-0.000946,0.000585,0.000361,-0.000721,0.001305,-0.002251,0.000361,0.000000,...,1,단일판매ㆍ공급계약체결,윤성에프앤씨는 2023년 12월 27일부터 2024년 11월 16일까지 10개월간 ...,단일판매ㆍ공급계약체결,0.088576,0.070284,0.062762,0.063064,0.064232,0.063511


# 방향 정합성

### 📘 모형식 정의

$$
\text{logit}^{-1}(x) = \frac{1}{1 + e^{-x}}
$$

$$
Pr(hit_{i,t} = 1) = \text{logit}^{-1}(\alpha + \beta \cdot period\_dummy_i)
$$
---

### 📊 변수 설명


이벤트 *i*, 창 *t* 에 대해 Delta CAR, post CAR 2개의 지표를 사용

$$
t \in \{10,\,20,\,\dots,\,60\}
$$

$$
\Delta CAR_{i,t} = CAR_{post,i,t} - CAR_{pre,i,t}  
$$

$$
CAR_{post,i,t}
$$

$$
label\_sign_i =
\begin{cases}
+1, & \text{긍정 (1)} \\
0, & \text{중립 (0)} \\
-1, & \text{부정 (-1)} 
\end{cases}
$$



$$
hit_{i,t} =
\begin{cases}
1, & \text{if } \operatorname{sign}(\Delta CAR_{i,t}) = label\_sign_i \\
0, & \text{otherwise}
\end{cases}
$$

---

### 📑 가설 설정

$$
\begin{aligned}
H_0 &: \beta = 0 \quad &(\text{도입 전후 차이 없음}) \\
H_1 &: \beta > 0 \quad &(\text{도입 후 정합률 상승})
\end{aligned}
$$

---

### 📈 해석

- delta CAR: β > 0 이고 **통계적으로 유의**하다면  → “텔레그램 서비스 출시 이후 과잉반응 감소 / 즉시 반영 후 빠른 수렴 → 정보의 선반영 및 사전 확산이 강화 → 정보 효율성 향상”을 의미한다고 볼 수 있다.
- Post CAR: β > 0 이고 **통계적으로 유의**하다면  → “텔레그램 서비스 출시 이후 이벤트 발표 이후의 정합률(정확도) 증가 → 시장 반응의 즉시 반영 강화 → 정보 효율성 향상”을 의미한다고 해석할 수 있다.


# 중립 포함

In [11]:
import sys
sys.path.append('..')
from hypothesis_test import (
    print_sample_summary_with_neutral,
    logistic_hit_delta_with_neutral_log,
    run_logistic_table
)

# === 전체 파이프라인 (중립 포함) ===

df = df_total.copy()
df["label_sign"] = df["label"]
df_total_sample = print_sample_summary_with_neutral(df, "label_sign")

# ΔCAR 기준 (pre/post window 차이 사용)
windows = [10, 20, 30, 40, 50, 60]
results_delta_with_neutral = run_logistic_table(
    logistic_hit_delta_with_neutral_log, df_total_sample, windows, label="ΔCAR"
)

=== Sample summary (including neutral) ===
Total events:        7,450
Neutrals:            2,087
  - Positive (1):    4,504
  - Negative (-1):   859


--- ΔCAR 기준 결과 ---
 t beta_star    std  t_stat      p_value  odds_ratio  p_before  p_after  diff_pp  n_obs     R2  adj_R2  neutral_epsilon
10 0.3393*** 0.0537   6.318 2.650000e-10       1.404     0.633    0.708     7.47   7450 0.0042  0.0038         0.006417
20 0.2812*** 0.0525   5.353 8.671100e-08       1.325     0.614    0.678     6.41   7450 0.0030  0.0026         0.006945
30 0.2119*** 0.0519   4.079 4.523764e-05       1.236     0.609    0.659     4.91   7450 0.0017  0.0013         0.007418
40   0.1148* 0.0517   2.222 2.625565e-02       1.122     0.618    0.644     2.67   7450 0.0005  0.0001         0.007978
50  0.1668** 0.0515   3.239 1.198034e-03       1.182     0.603    0.642     3.92   7450 0.0011  0.0007         0.008187
60   0.1203* 0.0515   2.336 1.947304e-02       1.128     0.611    0.639     2.82   7450 0.0006  0.0001        

# 중립 제거

In [13]:
import sys
sys.path.append('..')
from hypothesis_test import (
    print_sample_summary,
    logistic_hit_delta_log,
    run_logistic_table
)

# === 전체 파이프라인 ===

df = df_total.copy()
df["label_sign"] = df["label"]
df_nn = print_sample_summary(df, "label_sign")

# ΔCAR 기준 (pre/post window 차이 사용)
windows = [10, 20, 30, 40, 50, 60]
results_delta = run_logistic_table( logistic_hit_delta_log, df_nn, windows, label="ΔCAR", include_r2=False)


=== Sample summary (neutral removed) ===
Total events:        7,450
Removed neutrals:    2,087
Used (non-neutral):  5,363
  - Positive (1):    4,504
  - Negative (-1):   859


--- ΔCAR 기준 결과 ---
 t  beta_star    std  t_stat  p_value  odds_ratio  p_before  p_after  diff_pp  n_obs
10     0.0161 0.0613   0.262 0.793454       1.016     0.712    0.716     0.33   5363
20    -0.0129 0.0598  -0.215 0.829746       0.987     0.690    0.687    -0.28   5363
30    -0.0746 0.0591  -1.262 0.206986       0.928     0.683    0.666    -1.64   5363
40 -0.2015*** 0.0590  -3.414 0.000639       0.817     0.696    0.652    -4.42   5363
50   -0.1317* 0.0585  -2.251 0.024365       0.877     0.677    0.648    -2.95   5363
60   -0.1396* 0.0585  -2.387 0.016985       0.870     0.677    0.646    -3.12   5363
